In [5]:
from captcha.image import ImageCaptcha
import numpy as np
from PIL import Image
import random
import sys
import os
import tensorflow as tf

In [2]:
number = [str(i) for i in range(10)]
little = [chr(i) for i in range(97,123)]
large = [chr(i) for i in range(65,91)]
print(number)
print(little)
print(large)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


### 生成验证码

In [3]:
# 随机生成4个字符
def random_captcha_text(char_set=number,captcha_size=4):
    captcha_text = []
    for i in range(captcha_size):
        c = random.choice(char_set)
        captcha_text.append(c)
    return captcha_text

# 生成字符对应的验证码
def gen_captcha_text_and_image(path):
    image_ = ImageCaptcha()
    captcha_text = random_captcha_text()
    # 转换为字符串
    captcha_text = ''.join(captcha_text)
    captcha = image_.generate(captcha_text)
    image_.write(captcha_text, path + captcha_text + '.jpg')

path = 'captcha/images/'
num = 10000

if not os.path.exists(path):
    os.makedirs(path)
for i in range(num):
    gen_captcha_text_and_image(path)
    sys.stdout.write('\r>> Creating image %d/%d' % (i+1,num))
    sys.stdout.flush()
sys.stdout.write('\n')
sys.stdout.flush()

>> Creating image 10000/10000


### 生成tfrecord

In [10]:
_NUM_TEST = 500

_RANDOM_SEED = 2

DATA_DIR = 'captcha/images/'

TFRECORD_DIR = 'captcha/'

def _tfdata_exists(dataset_dir):
    for split_name in ['train','test']:
        tf_name = os.path.join(dataset_dir,split_name + '.tfrecord')
        if not tf.gfile.Exists(tf_name):
            return False
    return True

def _get_filenames(dataset_dir):
    captcha_names = []
    for filename in os.listdir(dataset_dir):
        path = os.path.join(dataset_dir, filename)
        captcha_names.append(path)
    return captcha_names

# def int64_feature(values):
#     if not isinstance(values, (tuple, list)):
#         values = [values]
#     return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def bytes_feature(values):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value = [values]))

def image_to_tfexample(image_data,label):
    return tf.train.Example(features=tf.train.Features(feature={
     'image': bytes_feature(image_data),
        'label':bytes_feature(label)
    }))

# 数据转换为record格式
def _convert_dataset(split_name, filenames, tfrecord_dir):
    assert split_name in ['train','test']
    
    with tf.Session() as sess:
        out_tfnames = os.path.join(tfrecord_dir, split_name + '.tfrecord')
        with tf.python_io.TFRecordWriter(out_tfnames) as tf_writer:
            for i,filename in enumerate(filenames):
                try:
                    sys.stdout.write('\r>> Converting image %d/%d' % (i+1,len(filenames)))
                    sys.stdout.flush()
                    
                    image_data = Image.open(filename)
                    image_data = image_data.resize((224,224))
                    image_data = image_data.convert('L').tobytes()
                    
                    # 拿到4位验证码并encode()
                    labels = filename.split('/')[-1].split('.')[0].encode()
                    example = image_to_tfexample(image_data, labels)
                    tf_writer.write(example.SerializeToString())
                
                except IOError as e:
                    print('Wrong: ' + filename)
                    print('Error: ',e)
                    print('Skip it\n')
    sys.stdout.write('\n')
    sys.stdout.flush()
    
if _tfdata_exists(TFRECORD_DIR):
    print('tf文件已存在')
else:
    photo_filenames = _get_filenames(DATA_DIR)
    
    random.seed(_RANDOM_SEED)
    # 打乱文件
    random.shuffle(photo_filenames)
    training_filenames = photo_filenames[_NUM_TEST:]
    testing_filenames = photo_filenames[:_NUM_TEST]
    
    _convert_dataset('train', training_filenames, TFRECORD_DIR)
    _convert_dataset('test', testing_filenames, TFRECORD_DIR)

>> Converting image 5831/5831
>> Converting image 500/500
